# Notebook 05 - YOLOv8 Detection Inference (Test Set)

This notebook loads the trained YOLOv8 model and performs plate detection on the test split.
Outputs include annotated images and a detection report.


In [3]:
!pip install -q ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.6 MB/s eta 0:00:00


In [4]:
from ultralytics import YOLO
print("Ultralytics imported successfully")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics imported successfully


In [5]:
!pip install -q ultralytics pandas opencv-python


In [6]:
from google.colab import drive
drive.mount("/content/drive")

import os,glob
import pandas as pd
from ultralytics import YOLO


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os,glob
import pandas as pd
from ultralytics import YOLO
print("Imports OK")


Imports OK


In [8]:
DATASET_DIR="/content/drive/MyDrive/ALPR_DATASET/Indian-License-Plate-1"
MODEL_PATH="/content/drive/MyDrive/ALPR_MODELS/plate_yolov8n_best.pt"

TEST_IMG_DIR=os.path.join(DATASET_DIR,"test","images")
OUT_DIR="/content/outputs_nb5"
os.makedirs(OUT_DIR,exist_ok=True)

print("MODEL exists:",os.path.exists(MODEL_PATH))
print("TEST images folder exists:",os.path.exists(TEST_IMG_DIR))


MODEL exists: True
TEST images folder exists: True


In [9]:
model=YOLO(MODEL_PATH)
print("Model loaded successfully")


Model loaded successfully


In [10]:
imgs=glob.glob(os.path.join(TEST_IMG_DIR,"*.jpg"))+glob.glob(os.path.join(TEST_IMG_DIR,"*.png"))+glob.glob(os.path.join(TEST_IMG_DIR,"*.jpeg"))
print("Total test images:",len(imgs))
print("Sample:",imgs[:3])


Total test images: 164
Sample: ['/content/drive/MyDrive/ALPR_DATASET/Indian-License-Plate-1/test/images/0c9ebe94-827d-4c74-9950-6816e70d1bab___IMG_8883-jpg_jpeg.rf.95bc1c7a0d210a19325143f42bb21161.jpg', '/content/drive/MyDrive/ALPR_DATASET/Indian-License-Plate-1/test/images/206c95ff-83b8-4273-b105-6637bf9a3038___numerix-number-plates-thodupuzha-idukki-number-plate-dealers-1lhwiga-jpg_jpeg.rf.92b8b8beb6b58360d6fc7a6085efa62f.jpg', '/content/drive/MyDrive/ALPR_DATASET/Indian-License-Plate-1/test/images/2c9306ab-3454-4ca0-89fd-db3f51dabcef___3e7fd381-0ae5-4421-8a70-279ee0ec1c61_Nissan-Terrano-front-three-quarters-on-plain-ground_jpg.rf.ae59e89690973515015208c309978350.jpg']


In [11]:
import cv2

rows=[]
for img_path in imgs:
    r=model.predict(img_path,conf=0.25,verbose=False)[0]

    # save annotated image
    out_path=os.path.join(OUT_DIR,os.path.basename(img_path))
    cv2.imwrite(out_path,r.plot())

    # log detections
    if r.boxes is not None and len(r.boxes)>0:
        for b in r.boxes:
            conf=float(b.conf[0])
            x1,y1,x2,y2=map(float,b.xyxy[0])
            rows.append([os.path.basename(img_path),conf,x1,y1,x2,y2])
    else:
        rows.append([os.path.basename(img_path),0.0,None,None,None,None])

print("Saved outputs to:",OUT_DIR)
print("Logged rows:",len(rows))


Saved outputs to: /content/outputs_nb5
Logged rows: 164


In [12]:
df=pd.DataFrame(rows,columns=["image","conf","x1","y1","x2","y2"])
csv_path=os.path.join(OUT_DIR,"detection_report.csv")
df.to_csv(csv_path,index=False)

print("Saved CSV:",csv_path)
df.head()


Saved CSV: /content/outputs_nb5/detection_report.csv


,image,conf,x1,y1,x2,y2
0,0c9ebe94-827d-4c74-9950-6816e70d1bab___IMG_888...,0.0,None,None,None,None
1,206c95ff-83b8-4273-b105-6637bf9a3038___numerix...,0.0,None,None,None,None
2,2c9306ab-3454-4ca0-89fd-db3f51dabcef___3e7fd38...,0.0,None,None,None,None
3,49bdf0d9-4e64-41eb-9c19-eabdc4afb051___Maruti-...,0.0,None,None,None,None
4,5cbd7465-ad12-4e6b-8eaf-d7056c3852f8___New-201...,0.0,None,None,None,None


In [13]:
det=df[df["conf"]>0]
print("Images:",len(imgs))
print("Detections:",len(det))
print("Mean conf:",det["conf"].mean() if len(det)>0 else 0)
print("Max conf :",det["conf"].max() if len(det)>0 else 0)
print("Min conf :",det["conf"].min() if len(det)>0 else 0)


Images: 164
Detections: 0
Mean conf: 0
Max conf : 0
Min conf : 0


In [14]:
img_path=imgs[0]
r=model.predict(img_path,conf=0.01,verbose=False)[0]
print("Boxes:",0 if r.boxes is None else len(r.boxes))


Boxes: 114


In [15]:
rows=[]
for img_path in imgs:
    r=model.predict(img_path,conf=0.05,verbose=False)[0]
    if r.boxes is not None and len(r.boxes)>0:
        for b in r.boxes:
            conf=float(b.conf[0])
            x1,y1,x2,y2=map(float,b.xyxy[0])
            rows.append([os.path.basename(img_path),conf,x1,y1,x2,y2])
    else:
        rows.append([os.path.basename(img_path),0.0,None,None,None,None])

print("Done. Logged rows:",len(rows))


Done. Logged rows: 167


In [16]:
rows=[]
for img_path in imgs:
    r=model.predict(img_path,conf=0.05,verbose=False)[0]

    if r.boxes is None or len(r.boxes)==0:
        rows.append([os.path.basename(img_path),0.0,None,None,None,None])
        continue

    b=r.boxes
    k=int(b.conf.argmax())
    conf=float(b.conf[k])
    x1,y1,x2,y2=map(float,b.xyxy[k])

    rows.append([os.path.basename(img_path),conf,x1,y1,x2,y2])

print("Done. Images processed:",len(rows))


Done. Images processed: 164


In [17]:
import pandas as pd,os
df=pd.DataFrame(rows,columns=["image","conf","x1","y1","x2","y2"])
csv_path=os.path.join(OUT_DIR,"detection_report_top1.csv")
df.to_csv(csv_path,index=False)
print("Saved:",csv_path)
df.head()


Saved: /content/outputs_nb5/detection_report_top1.csv


,image,conf,x1,y1,x2,y2
0,0c9ebe94-827d-4c74-9950-6816e70d1bab___IMG_888...,0.065987,155.357300,143.404343,269.281311,183.161942
1,206c95ff-83b8-4273-b105-6637bf9a3038___numerix...,0.000000,NaN,NaN,NaN,NaN
2,2c9306ab-3454-4ca0-89fd-db3f51dabcef___3e7fd38...,0.000000,NaN,NaN,NaN,NaN
3,49bdf0d9-4e64-41eb-9c19-eabdc4afb051___Maruti-...,0.060870,138.531143,167.110107,244.434250,214.933105
4,5cbd7465-ad12-4e6b-8eaf-d7056c3852f8___New-201...,0.069660,236.776108,250.269760,342.112274,302.816681


In [18]:
det=df[df["conf"]>0]
print("Images:",len(df))
print("Detected:",len(det))
print("Mean conf:",det["conf"].mean() if len(det)>0 else 0)
print("Max conf :",det["conf"].max() if len(det)>0 else 0)
print("Min conf :",det["conf"].min() if len(det)>0 else 0)


Images: 164
Detected: 58
Mean conf: 0.1000813625367551
Max conf : 0.22365084290504456
Min conf : 0.05002323165535927


In [19]:
import cv2,glob

for img_path in imgs:
    r=model.predict(img_path,conf=0.05,verbose=False)[0]
    out_path=os.path.join(OUT_DIR,os.path.basename(img_path))
    cv2.imwrite(out_path,r.plot())

print("Annotated outputs saved in:",OUT_DIR)


Annotated outputs saved in: /content/outputs_nb5
